In [ ]:
#儲存文章連結 正規表示法吃連結 用來建立連結資料
def getlinkv2 (script):
    global count
    global pkcount
    m = re.findall('(http://udn.com/news/.*\d+)',script)
    udn = 'udn'
    for link in m:
        num = str(pkcount)
        if pkcount < 10:
            num = str(0) * 4 + num
        elif pkcount < 100:     
            num = str(0) * 3 + num
        elif pkcount < 1000:     
            num = str(0) * 2 + num
        elif pkcount < 10000:     
            num = str(0) * 1 + num
        pk = udn + num   
        links = {'link':link,'area':area,'page':page,'count':count,'pk':pk}
        linkary.append(links)
        count +=1
        pkcount +=1

In [ ]:
#requests 方法 找出網頁特性 發現是jsp.net 直接對伺服器發送要求
import requests
import re
import time
#列出所需查詢區域 一開始先測試3區的效果
#['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
ary = ['中正區','大同區','中山區','松山區','大安區','萬華區','信義區','士林區','北投區','內湖區','南港區','文山區']
links = {'link':'','area':'','page':'','count':''}
linkary =[]
#每一區
pkcount = 1
for area in ary:
    count = 1
    headers = {
'Host':'search.udn.com',
'Referer':'http://udn.com/search/result/2/'+ area,
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
#每一頁    
    for page in range(1,13):
        res =requests.get('http://search.udn.com/search/searchResult4utf8.jsp?ch=udn.news2014&df=2&rc=15&wc=80&pw=220&mc=&q=' + \
                          area + '&fp={}'.format(page),headers = headers,timeout=3)
        time.sleep(3) #避免讓伺服器察覺攻擊
        script = res.text
        getlinkv2(script)

In [ ]:
import pandas as pd
data1 = pd.DataFrame(linkary)
data1

In [ ]:
#輸出成CSV檔將連結存起 假如需要可以讀回
data1.to_csv('udn_link.csv',encode='utf-8')

In [2]:
#讀回程式
import csv
linkary = []
with open('udn_linktst.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        linkary.append(row)

In [ ]:
# #擷取新聞內文 ptag
# def content(pcont):
#     cont = ''
#     for con in pcont:
#         cont = cont + con.text.strip()
#     return cont

In [16]:
#取得文章內容及標題
import requests
from bs4 import BeautifulSoup as bs
import pandas
import time
headers = {
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36'
}
for get in linkary:
    #強制取得 建議不要使用while 在一般requests套件中 會被伺服器端察覺
    text = ''
    while True:
        try:
            url = get['link']
            res = requests.get(url,headers=headers)
            soup = bs(res.content)
            break
        except:
            print 'try again 5sec'
            time.sleep(5)
        
        try:
            get['title'] = soup.select('#story_body_content h2')[0].text.strip()
            get['date'] = soup.select('#story_body_content h3')[0].text.strip().split(' ')[0]
            for p_tag in soup.select('#story_body_content p'):
                text += p_tag.text.strip()
            get['content'] = text
        
#     for article in soup.select('#story_body_content'):
#         try:
#             get['title'] = article.select('h2')[0].text.strip()
#             pcont = article.select('p')
#             get['content'] = content(pcont)
# #             get['content'] = article.select('p')[0].text.strip()
#             get['date'] = article.select('h3')[0].text.strip().split(' ')[0]
            
#             #get['time'] = article.select('h3')[0].text.strip().split(' ')[1] 發文時間可要可不要
        except:
            print 'this link has some problem'
            print get['link'], get['area'], get['count']
            continue

In [15]:
#將新聞內文轉成 DataFrame
import pandas as pd
data2 = pd.DataFrame(linkary)

#刪除多餘 column
udn_ok = data2.drop(data2.columns[[0]],axis = 1)
udn_ok

,area,count,link,page,pk
0,中正區,11,http://udn.com/news/story/1/1275914,1,udn00011
1,中正區,12,http://udn.com/news/story/7266/1274252,1,udn00012
2,中正區,13,http://udn.com/news/story/6/1273474,1,udn00013
3,中正區,14,http://udn.com/news/story/2/1273297,1,udn00014
4,中正區,15,http://udn.com/news/story/2/1273226,1,udn00015
5,中正區,16,http://udn.com/news/story/2/1273225,2,udn00016
6,中正區,17,http://udn.com/news/story/2/1273223,2,udn00017
7,中正區,18,http://udn.com/news/story/2/1273004,2,udn00018
8,中正區,19,http://udn.com/news/story/6655/1270657,2,udn00019
9,中正區,20,http://udn.com/news/story/7741/1270406,2,udn00020


In [11]:
#輸出完成品
udn_ok.to_csv('udn.csv',encoding='utf-8')

In [ ]:
# pcont = article.select('p')
# get['content'] = content(pcont)
# def content(pcont):
#     cont = ''
#     for con in pcont:
#         cont = cont + con.text.strip()
#     return cont